In [309]:
import tulip as tlp
from tulip.interfaces import omega as omega_int
from tulip import transys, abstract, spec, synth
from visualization import graph_builder as gb
import networkx as nx
from tulip.transys import machines
from GR1_defaults import settings

from tulip import dumpsmach
import pickle

In [310]:
path = 'var_inc/'

In [311]:
! rm -r var_inc

In [312]:
mkdir var_inc

In [313]:
have_bools = False

In [314]:
# Variables
if have_bools:
    env_vars = {'is_true':'boolean',
                'is_true2': 'boolean'}
else:
    env_vars = dict()
sys_vars = {'num':(0,3)}

In [315]:
increment = True

In [316]:
# Initialization
if have_bools:
    env_init = {'is_true', 'is_true2'}
else: 
    env_init = set()

if increment:
    sys_init = {'num=0'}
else:
    sys_init = {'num=3'}

In [317]:
# Safety
if have_bools:
    env_safe = {'(!is_true) -> X(is_true)',
                '(is_true) -> !X(is_true)',
                'X is_true -> X is_true2' # does not count initial state
    }
else:
    env_safe = set()

    
if increment:
    sys_safe = {'(num!=3) -> (X(num)=num+1)',
                '(num=3) -> (X(num)=3)'}
else:
    sys_safe = {'(num!=0) -> (X(num)=num-1)',
                '(num=0) -> (X(num)=0)'}



In [318]:
# Progress
env_prog = set()
if increment:
    sys_prog = {'num=3'}
else:
    sys_prog = {'num=0'}

In [319]:
specs = settings.set_specs(env_vars, sys_vars, env_init, sys_init,
                            env_safe, sys_safe, env_prog, sys_prog)

In [320]:
print(specs.pretty())

ENVIRONMENT VARIABLES:
	(none)

SYSTEM VARIABLES:
	num	(0, 3)

FORMULA:
ASSUMPTION:
GUARANTEE:
    INITIAL
	  (num=0)
    SAFETY
	  []((num!=3) -> (X(num)=num+1))
	& []((num=3) -> (X(num)=3))
    LIVENESS
	  []<>(num=3)



In [321]:
spec = tlp.synth._spec_plus_sys(specs, None, None, False, False)

In [322]:
# Automaton class found in omega/omega/symbolic/temporal.py
aut = omega_int._grspec_to_automaton(spec)

In [323]:
# Synthesizing system controller
ctrl = tlp.synth.synthesize(specs)
assert ctrl is not None, 'unrealizable'

removed 0 nodes from 5 total


In [324]:
 # Graphing
filename = path + "graph"
attributes = ['color', 'shape']

In [325]:
# Making a graph of the asynchronous GR(1) game with deadends.
g0 = gb.game_graph(aut, env='env', sys='sys', remove_deadends=False, qinit=aut.qinit)
h0 = gb._game_format_nx(g0, attributes)
pd0 = nx.drawing.nx_pydot.to_pydot(h0)
pd0.write_pdf(path + 'game.pdf')
# with open(filename, "wb") as file:
#     pickle.dump(g0, file)